# 04_EXPLORE_VINTAGE_CURVES.ipynb
================================

Objetivo: Explorar y validar el datamart de VINTAGE CURVES
- ¿Cuál es la estructura de datos de cohortes?
- ¿Cuánto tarda cada cohort en pagar su préstamo?
- ¿Cómo evoluciona el recovery rate por mes?
- ¿Hay diferencias por banda de riesgo?
- ¿Cuál es el patrón de evolución de ingresos?

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

In [ ]:
# Cargar datos desde exports
base_path = Path.cwd().parent.parent if 'data_validation' in str(Path.cwd()) else Path.cwd()
data_path = base_path / 'data' / 'exports'

print(f"📁 Working dir: {Path.cwd()}")
print(f"📂 Base path: {base_path}")
print(f"📍 Data path: {data_path}")

# Cargar vintage curves
vintage_curves = pd.read_excel(data_path / 'vintage_curves.xlsx')

print(f"\n✅ Vintage Curves cargadas: {len(vintage_curves):,} registros")

In [ ]:
print("="*80)
print("📊 DATASET: VINTAGE CURVES (Cohort Analysis)")
print("="*80)

# 1. ¿Cuántos registros hay?
print(f"\n1️⃣ Total de registros: {len(vintage_curves):,}")
print(f"   (Cada registro = 1 snapshot de cohort a cierta edad en meses)")

# 2. ¿Qué columnas tengo?
print(f"\n2️⃣ Columnas disponibles ({len(vintage_curves.columns)}):")
for i, col in enumerate(vintage_curves.columns, 1):
    print(f"   {i:2d}. {col}")

# 3. Ver primeros registros
print(f"\n3️⃣ Primeros 10 registros:")
display(vintage_curves.head(10))

# 4. Tipos de datos
print(f"\n4️⃣ Tipos de datos:")
print(vintage_curves.dtypes)

# 5. Valores nulos
print(f"\n5️⃣ Valores nulos:")
nulls = vintage_curves.isnull().sum()
nulls_pct = (nulls / len(vintage_curves) * 100).round(2)
null_df = pd.DataFrame({'Nulls': nulls, 'Percentage': nulls_pct})
print(null_df[null_df['Nulls'] > 0])
if null_df[null_df['Nulls'] > 0].empty:
    print("   ✅ Sin valores nulos")

In [ ]:
# 6. Estructura de COHORTES
print(f"\n6️⃣ ESTRUCTURA DE COHORTES:")

if 'vintage_month' in vintage_curves.columns:
    n_vintage_months = vintage_curves['vintage_month'].nunique()
    print(f"   - Meses de vintage: {n_vintage_months}")
    print(f"   Vintage months disponibles:")
    print(vintage_curves['vintage_month'].unique())
else:
    print("   ⚠️ No hay columna 'vintage_month'")

# Encontrar columna de edad/meses desde originación
age_cols = [col for col in vintage_curves.columns if any(x in col.lower() for x in ['month', 'age', 'days'])]
print(f"\n   - Posibles columnas de edad: {age_cols}")

if age_cols:
    age_col = [c for c in age_cols if 'since' in c.lower() or 'origination' in c.lower()]
    if not age_col:
        age_col = [age_cols[0]]
    age_col = age_col[0]
    
    print(f"\n   Usando columna de edad: '{age_col}'")
    n_ages = vintage_curves[age_col].nunique()
    max_age = vintage_curves[age_col].max()
    print(f"   - Edades únicas: {n_ages}")
    print(f"   - Edad máxima (meses): {max_age}")
    print(f"   - Rango de edades: {sorted(vintage_curves[age_col].unique())}")

# 7. Segmentos de riesgo
risk_cols = [col for col in vintage_curves.columns if 'risk' in col.lower() or 'segment' in col.lower()]
if risk_cols:
    risk_col = risk_cols[0]
    print(f"\n   - Columna de riesgo: '{risk_col}'")
    print(f"   Segmentos disponibles:")
    print(vintage_curves[risk_col].unique())
else:
    print(f"\n   ⚠️ No hay columna de riesgo/segmento")

In [ ]:
# 8. EJEMPLO: Seguir una cohort específica
print(f"\n8️⃣ EJEMPLO: Seguimiento de una COHORT específica")

if 'vintage_month' in vintage_curves.columns and age_col:
    # Tomar la primera cohort
    first_vintage = sorted(vintage_curves['vintage_month'].unique())[0]
    cohort_data = vintage_curves[vintage_curves['vintage_month'] == first_vintage].sort_values(age_col)
    
    print(f"\n   Cohort: {first_vintage}")
    print(f"   Registros para esta cohort: {len(cohort_data)}")
    
    # Mostrar la evolución
    cols_to_show = [col for col in cohort_data.columns if any(x in col.lower() for x in ['age', 'month', 'since', 'principal', 'revenue', 'recovery', 'default'])]
    if not cols_to_show:
        cols_to_show = cohort_data.columns[:10]
    
    print(f"\n   Evolución de la cohort a través del tiempo:")
    display(cohort_data[cols_to_show].head(10))
    print(f"\n   💡 Interpretación:")
    print(f"   - Cada fila = snapshot de la cohort en un mes específico")
    print(f"   - Las métricas muestran evolución acumulada desde originación")
    print(f"   - Permite ver cómo diferentes cohortes se desempeñan")

In [ ]:
# 9. METRICAS DE PERFORMANCE
print(f"\n9️⃣ METRICAS DE PERFORMANCE ACUMULADAS:")

# Buscar columnas de recovery/default
recovery_cols = [col for col in vintage_curves.columns if 'recovery' in col.lower() or 'principal' in col.lower()]
default_cols = [col for col in vintage_curves.columns if 'default' in col.lower()]
revenue_cols = [col for col in vintage_curves.columns if 'revenue' in col.lower()]

print(f"   - Columnas de recovery/principal: {recovery_cols}")
print(f"   - Columnas de default: {default_cols}")
print(f"   - Columnas de revenue: {revenue_cols}")

if recovery_cols:
    recovery_col = recovery_cols[0]
    print(f"\n   📊 {recovery_col}:")
    print(f"   - Mínimo: {vintage_curves[recovery_col].min():.4f}")
    print(f"   - Máximo: {vintage_curves[recovery_col].max():.4f}")
    print(f"   - Promedio: {vintage_curves[recovery_col].mean():.4f}")

if default_cols:
    default_col = default_cols[0]
    print(f"\n   📊 {default_col}:")
    print(f"   - Mínimo: {vintage_curves[default_col].min():.4f}")
    print(f"   - Máximo: {vintage_curves[default_col].max():.4f}")
    print(f"   - Promedio: {vintage_curves[default_col].mean():.4f}")

if revenue_cols:
    revenue_col = revenue_cols[0]
    print(f"\n   📊 {revenue_col}:")
    print(f"   - Total: ${vintage_curves[revenue_col].sum():,.2f}")
    print(f"   - Promedio: ${vintage_curves[revenue_col].mean():,.2f}")

In [ ]:
# 10. COMPARACIÓN POR BANDA DE RIESGO
print(f"\n🔟 PERFORMANCE POR BANDA DE RIESGO:")

if risk_col and recovery_cols:
    # Para cada banda de riesgo, ver stats agregadas
    risk_summary = vintage_curves.groupby(risk_col).agg({
        recovery_col: ['min', 'max', 'mean'],
        'total_loans_in_cohort': 'sum' if 'total_loans_in_cohort' in vintage_curves.columns else 'count'
    })
    print(risk_summary)
else:
    print("   ⚠️ Falta información de riesgo o recovery")

In [ ]:
# 11. ANALISIS DE VINTAGE (Cohorte más nueva vs más vieja)
print(f"\n1️⃣1️⃣ VINTAGE MÁS NUEVA vs MÁS VIEJA:")

if 'vintage_month' in vintage_curves.columns:
    vintages = sorted(vintage_curves['vintage_month'].unique())
    oldest_vintage = vintages[0]
    newest_vintage = vintages[-1]
    
    print(f"\n   Vintage MÁS VIEJA: {oldest_vintage}")
    oldest_data = vintage_curves[vintage_curves['vintage_month'] == oldest_vintage]
    print(f"   - Registros: {len(oldest_data)}")
    print(f"   - Max age: {oldest_data[age_col].max()} meses")
    if recovery_cols:
        print(f"   - Recovery rate final: {oldest_data[recovery_col].max():.4f}")
    
    print(f"\n   Vintage MÁS NUEVA: {newest_vintage}")
    newest_data = vintage_curves[vintage_curves['vintage_month'] == newest_vintage]
    print(f"   - Registros: {len(newest_data)}")
    print(f"   - Max age: {newest_data[age_col].max()} meses")
    if recovery_cols:
        print(f"   - Recovery rate final: {newest_data[recovery_col].max():.4f}")

In [ ]:
# 12. DISTRIBUCIÓN DE EDADES
print(f"\n1️⃣2️⃣ DISTRIBUCIÓN DE EDADES EN DATASET:")

if age_col:
    age_distribution = vintage_curves[age_col].value_counts().sort_index()
    print(f"   Registros por edad (meses):")
    print(age_distribution)
    
    print(f"\n   📊 Estructura:")
    print(f"   - Edades mínima: {age_distribution.index.min()} meses")
    print(f"   - Edades máxima: {age_distribution.index.max()} meses")
    print(f"   - Registros por edad: {age_distribution.mean():.1f} (promedio)")

In [ ]:
print("\n" + "="*80)
print("✅ CONCLUSIONES - VINTAGE CURVES")
print("="*80)

if 'vintage_month' in vintage_curves.columns:
    n_vintages = vintage_curves['vintage_month'].nunique()
    
    print(f"""
📊 ESTRUCTURA DE COHORTES:
   Total registros: {len(vintage_curves):,}
   Cohortes (vintage months): {n_vintages}
   Edad máxima (meses): {vintage_curves[age_col].max()}
   
📈 TIPO DE ANÁLISIS:
   - Permite analizar evolución de cada cohort a lo largo del tiempo
   - Cada fila = 1 snapshot de la cohort en cierta edad
   - Compara desempeño entre cohortes originadas en diferentes meses
   
⚠️ VALIDACIÓN:
   - Verificar que recovery rates sean monótonos (no decrecen con edad)
   - Verificar que cohortes más viejas tengan más edades que más nuevas
   - Comparar performance vs tabla de loans (¿son consistentes?)
""")
else:
    print("\n⚠️ Falta información para conclusiones")